In [ ]:
column_headers = [
    "Unique identifier",
    "Last name / organization",
    "First name",
    "Middle initial",
    "Credential ('', 'M.D.', 'D.O.', 'MD','DO', 'OTR', 'D.C.', ...)",
    "Gender",
    "Entity ('I' for individual, 'O' for organization)",
    "Address line 1",
    "Address line 2",
    "City",
    "Zip",
    "State",
    "Country",
    "Specialty ('Internal Medicine', 'Pathology', ...)",
    "Participates in medicare ('Y' or 'N')",
    "Place of service ('F' for facility or 'O' for other)",
    "HCPCS code (a procedure code)",
    "HCPCS code description",
    "HCPCS drug indicator ('Y' or 'N')",
    "Line service count (people, hours, miles, ...)",
    "Beneficiary unique count (number of distinct beneficiary, possibly receiving many procedures)",
    "Beneficiary day service count (number of distinct procedures, possibly on the same person)",
    "Average medicare allowed amount",
    "Standard deviation",
    "Average submitted charges amount",
    "Standard deviation",
    "Average medicare payment amount",
    "Standard deviation"
]
for i in range(0, len(column_headers)):
    print str(i) + ": " + column_headers[i]

In [ ]:
# Computing statistics, building dictionaries:

from math import floor

procedure_dictionary = dict()
procedure_count = dict()
procedure_total_submitted = dict()
procedure_total_allowed = dict()

infile = open("provider_utilization_2013.txt","r")
line = infile.readline()
line = infile.readline()
#for c in range(10000):
while True:   
    line = infile.readline()
    if(line == ""):
        break
    tokens = line.split("\t")
    if tokens[11] == "MA":
        procedure_id = tokens[16]

        procedure_dictionary[procedure_id] = tokens[17]

        number_of_procedures = int(tokens[21])
        avg_allowed_charge = float(tokens[22])
        avg_submitted_charge = float(tokens[24])
    
        if procedure_id not in procedure_count:
            procedure_count[procedure_id] = number_of_procedures
            procedure_total_submitted[procedure_id] = avg_submitted_charge * number_of_procedures
            procedure_total_allowed[procedure_id] = avg_allowed_charge * number_of_procedures
        else:
            procedure_count[procedure_id] += number_of_procedures
            procedure_total_submitted[procedure_id] += avg_submitted_charge * number_of_procedures
            procedure_total_allowed[procedure_id] += avg_allowed_charge * number_of_procedures
infile.close()

procedure_avg_allowed = dict()
procedure_avg_submitted = dict()
for procedure_id in procedure_count:
    procedure_avg_submitted[procedure_id] = procedure_total_submitted[procedure_id] / procedure_count[procedure_id]
    procedure_avg_allowed[procedure_id] = procedure_total_allowed[procedure_id] / procedure_count[procedure_id]

    
# procedure_distribution = dict()
# infile = open("provider_utilization_2013.txt","r")
# line = infile.readline()
# line = infile.readline()
# #for c in range(10000):
# while True:
#     line = infile.readline()
#     if(line == ""):
#         break
#     tokens = line.split("\t")
    
#     procedure_id = tokens[16]
#     number_of_procedures = int(tokens[21])
#     avg_submitted_charge = float(tokens[24])
#     state = tokens[11]
    
#     bin_size = procedure_avg_allowed[procedure_id]
    
#     if (procedure_id, state) not in procedure_distribution:
#         procedure_distribution[(procedure_id, state)] = [0 for i in range(8)]
        
#     i = int(floor(avg_submitted_charge / bin_size))
#     if i > 7:
#         i = 7
#     procedure_distribution[(procedure_id, state)][i] += number_of_procedures
    
# infile.close()

In [ ]:
# Load procedure-related mySQL tables 

reset = True;
year = 2013

import os, django
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "medissect.settings")
django.setup()
from explorer.models import ProcedureDescriptor, ProcedureAvgCharges

if reset:
    ProcedureDescriptor.objects.all().delete()
    
for code in procedure_dictionary:
    if(ProcedureDescriptor.objects.filter(code = code).count() == 0):
        descriptor = ProcedureDescriptor(code = code, descriptor = procedure_dictionary[code])
        descriptor.save()    

if reset:
    ProcedureAvgCharges.objects.all().delete()

for code in procedure_dictionary:
    descriptor = ProcedureDescriptor.objects.get(code = code)
    charges = ProcedureAvgCharges(
        descriptor = descriptor,
        year = year,
        count = procedure_count[code],
        allowed = procedure_avg_allowed[code],
        submitted = procedure_avg_submitted[code])
    charges.save()
    

#for pair in procedure_distribution:
#    (code, state) = pair
#    descriptor = ProcedureDescriptor.objects.get(code = code)
#    i = 0
#    for value in procedure_distribution[pair]:
#        counts = ProcedureCounts(
#            descriptor = descriptor, 
#            year = year,
#            state = state,
#            index = i,
#            value = value)
#        counts.save()
#        i += 1

In [ ]:
year = 2013
reset = True

import django
import os
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "medissect.settings")
django.setup()
from explorer.models import Procedure, ProcedureDescriptor, Provider
from django.db import IntegrityError

if reset:
    Procedure.objects.all().delete()

infile = open("provider_utilization_2013.txt","r")
line = infile.readline()
line = infile.readline()
# for c in range(10000):
while True:
    line = infile.readline()
    if(line == ""):
        break
    tokens = line.split("\t")
    
    procedure_id = tokens[16]
    provider_id = int(tokens[0])
    procedure_count = int(tokens[21])
    avg_submitted_charge = float(tokens[24])
    state = tokens[11]
    if state != "MA":
        continue
    existing = Procedure.objects.filter(year = year);
    existing = existing.filter(provider__npi = provider_id);
    existing = existing.filter(descriptor__code = procedure_id);
    if(existing.count() == 0):
        try:
            descriptor = ProcedureDescriptor.objects.get(code = procedure_id)
        except ProcedureDescriptor.DoesNotExist:
            print "Missing descriptor"
            continue

        try:
            provider = Provider.objects.get(npi = provider_id)
        except Provider.DoesNotExist:
            print "Missing provider"
            continue

        procedure = Procedure(
            descriptor = descriptor,
            provider = provider,
            year = year,
            procedure_count = int(tokens[21]),
            beneficiary_count = int(tokens[20]),
            line_service_count = float(tokens[19]),
            allowed_avg = float(tokens[22]),
            allowed_std = float(tokens[23]),
            submitted_avg = float(tokens[24]),
            submitted_std = float(tokens[25]),
            payed_avg = float(tokens[26]),
            payed_std = float(tokens[27]),
        )
    else:
        n1 = procedure.procedure_count
        n2 = int(tokens[21])
        
        procedure = existing[0]
        procedure.procedure_count += n2
        procedure.beneficiary_count += int(tokens[20])
        procedure.line_service_count += float(tokens[19])
        procedure.allowed_avg = (n1 * procedure.allowed_avg + n2 * float(tokens[22]))/ (n1 + n2)
        procedure.submitted_avg = (n1 * procedure.submitted_avg + n2 * float(tokens[24]))/ (n1 + n2)
        procedure.payed_avg = (n1 * procedure.payed_avg + n2 * float(tokens[26]))/ (n1 + n2)
        
    procedure.save()
        
infile.close()

In [ ]:
import django
import os
import re
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "medissect.settings")
django.setup()
from explorer.models import Procedure, ProcedureDescriptor, Provider, Location

providers = Provider.objects.all();
coords = set([(p.latitude, p.longitude) for p in providers]);
for coord in coords:
    p = providers.filter(latitude = coord[0]).filter(longitude = coord[1])[0]
    loc = Location(
        street = p.street1,
        city = p.city,
        zipcode = p.zipcode,
        state = p.state,
        country = p.country,
        longitude = p.longitude,
        latitude = p.latitude
    )
    loc.save()

In [ ]:
import django
import os
import re
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "medissect.settings")
django.setup()
from explorer.models import ProcedureDescriptor, OldProvider, Location, Provider

providers = OldProvider.objects.all();
for p in providers:
    loc = Location.objects.filter(latitude = p.latitude).filter(longitude = p.longitude);
    if(loc.count() == 1):
        np = Provider(
            npi = p.npi,
            last_name = p.last_name,
            first_name = p.first_name,
            middle_initial = p.middle_initial,
            credentials = p.credentials,
            gender = p.gender,
            is_organization = p.is_organization,
            location = loc[0],
            street2 = p.street2,
            medicare_participant = p.medicare_participant,
            at_facility = p.at_facility,
            expensiveness = p.expensiveness
        )
        np.save()
    else:
        print "error"
    

In [ ]:
import django
import os
import re
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "medissect.settings")
django.setup()
from explorer.models import Procedure, ProcedureDescriptor, Provider

providers = Provider.objects.all();
for p in providers:
    sub_street = p.street1
    sub_street = re.sub(r'( STREET[,.]*$)|( STR[,.]*$)|( ST[,.]*$)', ' ST', sub_street)
    sub_street = re.sub(r'( DRIVE[,.]*$)|( DRV[,.]*$)|( DR[,.]*$)', ' DR', sub_street)
    sub_street = re.sub(r'( AVENUE[,.]*$)|( AVE[,.]*$)', ' AVE', sub_street)
    sub_street = re.sub(r'( ROAD[,.]*$)|( RD[,.]*$)', ' RD', sub_street)
    sub_street = re.sub(r'( HIGHWAY[,.]*$)|( HWY[,.]*$)', ' HWY', sub_street)
    sub_street = re.sub(r'( LANE[,.]*$)|( LN[,.]*$)', ' LN', sub_street)
    sub_street = re.sub(r'( PLACE[,.]*$)|( PL[,.]*$)', ' PL', sub_street)
    sub_street = re.sub(r'( PARKWAY[,.]*$)|( PKWY[,.]*$)', ' PKWY', sub_street)
    sub_street = re.sub(r'( TURNPIKE[,.]*$)|( TPKE[,.]*$)', ' TPKE', sub_street)
    sub_street = re.sub(r'( TERRACE[,.]*$)|( TER[,.]*$)', ' TER', sub_street)
    sub_street = re.sub(r'( SQUARE[,.]*$)|( SQ[,.]*$)', ' SQ', sub_street)
    sub_street = re.sub(r'( BOULEVARD[,.]*$)|( BLVD[,.]*$)', ' BLVD', sub_street)
    
    sub_street = re.sub(r'( SAINT )|( ST[,.]* )', ' ST ', sub_street)
    sub_street = re.sub(r'(^SAINT )|(^ST[,.]* )', 'ST ', sub_street)
    
    sub_street = re.sub(r'( MOUNT )|( MT[,.]* )', ' MOUNT ', sub_street)
    sub_street = re.sub(r'(^MOUNT )|(^MT[,.]* )', 'MOUNT ', sub_street)
    
    sub_street = re.sub(r'( AVENUE )', ' AVE ', sub_street)
    
    sub_street = re.sub(r'( N[,.]* )', ' NORTH ', sub_street)
    sub_street = re.sub(r'( S[,.]* )', ' SOUTH ', sub_street)
    sub_street = re.sub(r'( W[,.]* )', ' WEST ', sub_street)
    sub_street = re.sub(r'( E[,.]* )', ' EAST ', sub_street)
    
    sub_street = re.sub(r'( N[,.]*$)', ' NORTH', sub_street)
    sub_street = re.sub(r'( S[,.]*$)', ' SOUTH', sub_street)
    sub_street = re.sub(r'( W[,.]*$)', ' WEST', sub_street)
    sub_street = re.sub(r'( E[,.]*$)', ' EAST', sub_street)
    
    if p.street1 != sub_street:
        p.street1 = sub_street
        p.save()
#         print p.street1
#         print sub_street
#         print " "

In [ ]:
import django
import os
import re
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "medissect.settings")
django.setup()
from explorer.models import Procedure, ProcedureDescriptor, Provider

providers = Provider.objects.all();
ls = set();
digits = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9');
for p in providers:
    if p.street1[0] not in digits and len(p.street2) > 0 and p.street2[0] in digits:
        swap = p.street1
        p.street1 = p.street2
        p.street2 = swap
        p.save()
#         print p.street1
#         print p.street2
#         print ""
# for x in ls:
#     print x